In [ ]:
import pandas as pd
import scipy.stats
import os
import numpy as np

In [ ]:
def format_status(status, rel_err):
    if np.isnan(rel_err):
        return "error"
    else:
        return status

def format_err(status, rel_err):
    if status == "success":
        return f"G[{rel_err:.2f}]"
    elif status == "mismatch":
        return f"R[{rel_err:.2f}]"
    else:
        return "ERROR"

def preprocess(name, logdir="logs"):
    f= next(f for f in os.listdir(logdir) if f"_{name}_" in f)
    df = pd.read_csv(f"{logdir}/{f}", index_col=0)
    df["status"] = df.apply(lambda x: format_status(x.status, x.rel_err), axis=1)
    df["err"] = df.apply(lambda x: format_err(x.status, x.rel_err), axis=1)
    df.columns = [f"{name}_{x}" for x in df.columns]
    df

    return df

In [ ]:
bnaf = preprocess("AutoBNAFNormal")
delta = preprocess("AutoDelta")
diag = preprocess("AutoDiagonalNormal")
iaf = preprocess("AutoIAFNormal")
laplace = preprocess("AutoLaplaceApproximation")
lowrank = preprocess("AutoLowRankMultivariateNormal")
multi = preprocess("AutoMultivariateNormal")
auto = preprocess("AutoNormal")

res = pd.concat([bnaf, delta, diag, iaf, laplace, lowrank, multi, auto], axis=1)

In [ ]:
res['example'] = res.index.map(lambda x: x.split("-")[1])
res['data'] = res.index.map(lambda x: x.split("-")[0])
res = res.sort_values(by='example')
res = res.drop(["mcycle_gp-accel_gp", "diamonds-diamonds","gp_pois_regr-gp_pois_regr", "gp_pois_regr-gp_regr", "hudson_lynx_hare-lotka_volterra", "one_comp_mm_elim_abs-one_comp_mm_elim_abs"])


In [ ]:
status = res[[
    "AutoBNAFNormal_status",
    "AutoDelta_status",
    "AutoDiagonalNormal_status",
    "AutoIAFNormal_status",
    "AutoLaplaceApproximation_status",
    "AutoLowRankMultivariateNormal_status",
    "AutoMultivariateNormal_status",
    "AutoNormal_status"
]].apply(pd.value_counts)

In [ ]:

print(
    res[
        [
            "AutoBNAFNormal_err",
            "AutoDelta_err",
            "AutoDiagonalNormal_err",
            "AutoIAFNormal_err",
            "AutoLaplaceApproximation_err",
            "AutoLowRankMultivariateNormal_err",
            "AutoMultivariateNormal_err",
            "AutoNormal_err"
        ]
    ].to_latex()
    .replace("G[", "\Green{")
    .replace("R[", "\Red{")
    .replace("]", "}")
    .replace("ERROR", "\emark")
)

In [ ]:
print(status.to_latex())

In [ ]:
avg = res[[
            "AutoBNAFNormal_rel_err",
            "AutoDelta_rel_err",
            "AutoDiagonalNormal_rel_err",
            "AutoIAFNormal_rel_err",
            "AutoLaplaceApproximation_rel_err",
            "AutoLowRankMultivariateNormal_rel_err",
            "AutoMultivariateNormal_rel_err",
            "AutoNormal_rel_err"
        ]].mean(axis=0)
avg = avg.apply(lambda x: f"{x:.2f}")

In [ ]:
print(pd.DataFrame([avg.to_list()], columns=list(avg.index), index=["Average"]).to_latex())